In [ ]:
# INFERENCE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor,ToPILImage,Normalize
from utils.dataset import LoadCoCoDataset
from utils.model import construct_style_loss_model,construct_decoder_from_encoder
from utils.losses import content_gatyes,style_gatyes,style_mmd_polynomial,adaIN
from utils.utility import video_to_frame_generator,video_to_frames,normalize
import cv2
import os
from torchvision.datasets import CocoDetection
from copy import deepcopy
import torch.utils.data as data
from torchvision.models import vgg19,VGG19_Weights
from PIL import Image,ImageOps
import torch.optim as optim
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [ ]:
# INFERENCE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On device: ", device)

In [ ]:
# INFERENCE

IMAGE_SIZE = (256,256)

In [ ]:
# INFERENCE

CONTENT_LAYERS = [6]
STYLE_LAYERS = [9,12,14]
CONTENT_LAYERS_WEIGHTS = [1.0]
STYLE_LAYERS_WEIGHTS = [1.0,1.0,1.0]

In [ ]:
# INFERENCE

# load standard vgg19 model
vgg = vgg19(VGG19_Weights.DEFAULT)

# remove classification head
vgg = vgg.features

# prepend a normalization layer
vgg = nn.Sequential(Normalize(mean = (0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)), *vgg)

style_loss_model = construct_style_loss_model(vgg,CONTENT_LAYERS,STYLE_LAYERS)
style_loss_model = style_loss_model.eval()
style_loss_model.requires_grad_(False)
style_loss_model = style_loss_model.to(device)

# as an encoder for our feedforward model we will just use the already existing vgg layers up to some point
vgg_encoder = vgg[:8]

# based on that we build a decoder that reverses our encoder and matches the shapes through interpolation
decoder = construct_decoder_from_encoder(vgg_encoder.cpu(),3,*IMAGE_SIZE)

In [ ]:
COCO_PATH = "./test2017/"
STYLE_IMAGE_PATH = "./wave.jpg"
BATCH_SIZE = 8

In [ ]:
coco_dataloader = torch.utils.data.DataLoader(LoadCoCoDataset(COCO_PATH, BATCH_SIZE ,IMAGE_SIZE),batch_size=BATCH_SIZE)
style_img = Image.open(STYLE_IMAGE_PATH).convert('RGB')
# center crop image
style_img = ImageOps.fit(style_img,(min(style_img.size),min(style_img.size))).resize(IMAGE_SIZE)
style_img = ToTensor()(style_img).permute(0,2,1).unsqueeze(0).repeat(BATCH_SIZE,1,1,1).to(device)

In [ ]:
# INFERENCE

style_model = nn.Sequential(vgg_encoder,decoder).to(device)
optimimizer = optim.Adam(decoder.parameters(),lr=0.0001)

In [ ]:
# INFERENCE

EPOCHS = 10000
ACCUM_GRAD = 1
PRINT_STATS_EVERY = 500
STYLE_WEIGHT = 10000.0
LOSS_CONTENT = content_gatyes
LOSS_STYLE = style_gatyes
SAVE_PATH = "./feedforward_save/"

In [ ]:
os.makedirs(SAVE_PATH,exist_ok = True)

content_loss_aggregator = []
style_loss_aggregator = []

for epoch in tqdm(range(EPOCHS)):

    optimimizer.zero_grad()

    # Since we might have to work with small batch_sizes, simulate big batch sizes by accumulating gradients over multiple batches
    for _ in range(ACCUM_GRAD):
        content_img = next(iter(coco_dataloader)).to(device)

        with torch.no_grad():
            _,content_features_target,_ = style_loss_model((content_img,[],[]))
            _,_,style_features_target = style_loss_model((style_img,[],[]))
        

        prediction = style_model(content_img)

        # get the features from the content and style layers of our prediction
        _,content_features,style_features= style_loss_model((prediction,[],[]))

        content_loss = 0.0
        for f,f_target,weight in zip(content_features,content_features_target, CONTENT_LAYERS_WEIGHTS):
            content_loss += weight*LOSS_CONTENT(f,f_target).mean()

        style_loss = 0.0
        for f,f_target,weight in zip(style_features,style_features_target, STYLE_LAYERS_WEIGHTS):
            style_loss += weight*LOSS_STYLE(f,f_target).mean()

        style_loss *= STYLE_WEIGHT

        content_loss_aggregator.append(content_loss.detach().cpu())
        style_loss_aggregator.append(style_loss.detach().cpu())

        loss = content_loss+style_loss

        loss.backward()

    optimimizer.step()

    # Every PRINT_STATS_EVERY epochs we want to print the current performance, save the weights, and also save an example prediction
    if epoch%PRINT_STATS_EVERY == 0 and not epoch == 0:
        print("Ending epoch: ", str(epoch), " with content loss: ", torch.stack(content_loss_aggregator).mean().numpy().item(),  " and style loss: ", torch.stack(style_loss_aggregator).mean().numpy().item())
        img = next(iter(coco_dataloader))[0]
        ToPILImage()(img.permute(0,2,1)).save(SAVE_PATH + "example_input_model.jpg")
        img = img.unsqueeze(0).to(device)

        with torch.no_grad():   
            img = style_model(img)
        img = ToPILImage()(img.cpu().squeeze(0).permute(0,2,1))
        img.save(SAVE_PATH + "example_output_model.jpg")
        torch.save(style_model.state_dict(), SAVE_PATH + "model_weights.pth")
        torch.save(optimimizer.state_dict(), SAVE_PATH + "optim_weights.pth")


    
    

In [11]:
# INFERENCE

style_model.load_state_dict(torch.load(SAVE_PATH + "model_weights.pth",map_location=device))


<All keys matched successfully>

In [12]:
# INFERENCE

CONTENT_IMAGE_PATH = "./dragon.jpg"

In [13]:
# INFERENCE

content_img = Image.open(CONTENT_IMAGE_PATH).convert('RGB')
# center crop image
content_img = ImageOps.fit(content_img,(min(content_img.size),min(content_img.size))).resize(IMAGE_SIZE)
content_img = ToTensor()(content_img).permute(0,2,1).to(device)

In [14]:
# INFERENCE

content_style_image = style_model(content_img.unsqueeze(0)).squeeze(0).cpu()
content_style_image = ToPILImage()(content_style_image.permute(0,2,1))
content_style_image.show()